<a href="https://colab.research.google.com/github/hilmirzd/sandbox-superstore-MySkill/blob/main/Hilmi's_Python_Porto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Hello, World :) project by HILMIRZD

In [ ]:
# Masih dalam bentuk data warehouse di URL

# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.offsets import BDay

In [ ]:
# Panggil sumber data dari URL
path_od = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/order_detail.csv"
path_pd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/payment_detail.csv"
path_cd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/customer_detail.csv"
path_sd = "https://raw.githubusercontent.com/dataskillsboost/FinalProjectDA11/main/sku_detail.csv"
df_od = pd.read_csv(path_od)
df_pd = pd.read_csv(path_pd)
df_cd = pd.read_csv(path_cd)
df_sd = pd.read_csv(path_sd)

In [ ]:
# Gabung data dengan SQL fungsi JOIN

# Call SQL
from sqlite3 import connect
conn = connect(':memory:')
df_od.to_sql('order_detail',conn, index=False, if_exists='replace')
df_pd.to_sql('payment_detail', conn, index=False, if_exists='replace')
df_sd.to_sql('sku_detail', conn, index=False, if_exists='replace')
df_cd.to_sql('customer_detail', conn, index=False, if_exists='replace')

# Jalankan fungsi JOIN
df = pd.read_sql("""
SELECT
    order_detail.*,
    payment_detail.payment_method,
    sku_detail.sku_name,
    sku_detail.base_price,
    sku_detail.cogs,
    sku_detail.category,
    customer_detail.registered_date
FROM order_detail
LEFT JOIN payment_detail
    on payment_detail.id = order_detail.payment_id
LEFT JOIN sku_detail
    on sku_detail.id = order_detail.sku_id
LEFT JOIN customer_detail
    on customer_detail.id = order_detail.customer_id
""", conn)

In [ ]:
# Cek tipe data per kolom
df.dtypes

In [ ]:
# Cek data random sampling
df.sample(10)

In [ ]:
# Hygiene data

# Cek ada data null
df.isnull().sum()

In [ ]:
# Clear, cek tipe data

# Biar memudahkan perhitungan, ubah float ke integer pada before_discount, discount_amount, after_discount, dan base_price
df = df.astype({"before_discount":'int', "discount_amount":'int', "after_discount":'int',"base_price":'int'})
df.dtypes

In [ ]:
# Pada order_date dan registered_date masih dalam bentuk object, ubah ke datetime
df['order_date']= pd.to_datetime(df['order_date'])
df['registered_date']= pd.to_datetime(df['registered_date'])
df.dtypes

In [ ]:
# Data already hygiene
df.sample(5)

In [ ]:
# Download data untuk dimasukkan ke GitHub :)
df.to_csv('data_sql.csv', index=False)
from google.colab import files
files.download('data_sql.csv')

Setelah ini, lanjut analisa dengan case sbb:

Analisa 1
---
Mencari data penjualan terbanyak pada produk kategori “Mobiles & Tablets” selama tahun 2022.


In [ ]:
# Filter dasar
df_filtered = df[
    (df['is_valid'] == 1) & # filter yang valid
    (df['category'] == 'Mobiles & Tablets') & # filter berdasarkan kategory
    (df['order_date'].dt.year == 2022) # filter tahun
].copy() # biar tidak mengubah data asli

df_filtered.head(5)

In [ ]:
# Data sudah difilter, saat ini cari 5 teratas yang paling laris
top5 = (
    df_filtered
    .groupby('sku_name', as_index=False)['qty_ordered'] # cari nama produknya dan hitung qty nya, dalam bentuk kolom
    .sum()
    .sort_values(by='qty_ordered', ascending=False) # diurutkan
    .head(5)
    .reset_index(drop=True)
)

top5.insert(0, 'rank', range(1, len(top5) + 1)) # tambahkan urutan peringkat

top5

In [ ]:
# visualiskan data dengan seaborn dalam bentuk bar chart
plt.figure(figsize=(8,5))
sns.barplot(data=top5, x='qty_ordered', y='sku_name', palette='Blues_r')
plt.xlabel('Jumlah Terjual (qty)')
plt.ylabel('Produk')
plt.title('Top 5 Produk - Mobiles & Tablets (2022)')
plt.show() # hasil 5 produk terlaris di tahun 2022

Analisa 2
---
Melihat jumlah sisa dalam kategori "Others" pada tahun 2022, kemudian dianalisa data penjualanya apakah ada penurunan dari tahun sebelumnya.
Jika ada, dicari produk apa yang memiliki penurunan tertinggi pada kategori yang memiliki penurunan penjualan tertinggi.


In [ ]:
# Filter dasar
df_filtered = ( df[
    (df["order_date"].dt.year.isin([2021, 2022])) # cari barang yang terjual di tahun 2021 dan 2022
    & (df["is_valid"] == 1)
].copy())

df_filtered.head(5)

In [ ]:
# Cari data perkategori
qty_per_category_year = (
    df_filtered
    .groupby([df_filtered["order_date"].dt.year, "category"]) # filter dari kategori dan total order
    ["qty_ordered"]
    .sum()
    .reset_index()
    .rename(columns={ # bikin kolom
        "order_date": "year",
        "qty_ordered": "total_quantity"})
)

qty_per_category_year.head(5)

In [ ]:
# setelah didapatkan filter perkategori, lakukan pivot agara dapat dilihat pertahun pada kategori
qty_pivot = (
    qty_per_category_year
    .pivot(index="category",
           columns="year", # lihat per tahun 2022 dan 2021
           values="total_quantity")
    .fillna(0)
    .astype(int))
qty_pivot["selisih"] = qty_pivot[2022] - qty_pivot[2021] # cari selisihnya dari 2021 ke 2022 untuk mengetahui apakah terjadi penurunan
qty_pivot = qty_pivot.sort_values(by="selisih", ascending=True) # urutkan dari selisih tertinggi

# tampilkan pivotnya
from IPython.display import display # import pivot
display(qty_pivot)

In [ ]:
# buat visualisasi datanya
diff_sorted = qty_pivot["selisih"] # ambil kolom selisih untuk jadi metric
fig, ax = plt.subplots(figsize=(10, max(6, 0.3 * len(diff_sorted))))
colors = ["red" if val < 0 else "green" for val in diff_sorted.values]

ax.barh(diff_sorted.index, diff_sorted.values, color=colors)
ax.set_xlabel("Selisih dalam unit")
ax.set_title("Selisih Produk Terjual per Kategori 2021 sd 2022")
ax.invert_yaxis()
plt.tight_layout()
plt.show() # hasil penurunan tertinggi perkategori dari 2021 ke 2022

In [ ]:
# Hasil menunjukan "Others" memiliki penurunan tertinggi, sehingga perlu dicari produk apa yang memiliki penurunan tertinggi

# Filter dasar untuk data penjualan (menggunakan df yang sudah ada)
# Pastikan 'order_date' sudah dalam format datetime dan 'category' sudah ada
# Tambahkan kolom 'year' jika belum ada (cek dahulu sebelum menambahkan)
if 'year' not in df.columns:
    df['year'] = df['order_date'].dt.year

product_sales = (
    df[df["category"] == "Others"]
    .groupby([ "sku_name", "year"])["qty_ordered"]
    .sum()
    .reset_index()
)

# Filter untuk mengambil data penjualan 2021 dan 2022
sales_2021 = product_sales[product_sales["year"] == 2021][["sku_name", "qty_ordered"]].rename(columns={"qty_ordered": "Sales_2021"})
sales_2022 = product_sales[product_sales["year"] == 2022][["sku_name", "qty_ordered"]].rename(columns={"qty_ordered": "Sales_2022"})

# Filter untuk compare tahun 2021 dan 2022
compare_sales = sales_2021.merge(sales_2022, on="sku_name", how="inner")
compare_sales["Decrease"] = compare_sales["Sales_2021"] - compare_sales["Sales_2022"]
compare_sales["Decrease_%"] = (compare_sales["Decrease"] / compare_sales["Sales_2021"]) * 100

In [ ]:
# Filter hanya produk yang mengalami penurunan penjualan
mostdecrease = (
    compare_sales
    .sort_values(by="Decrease_%", ascending=False)
    .reset_index(drop=True)
)

# filter produk dengan tertinggi dalam persen (Decrease_% > 0)
mostdecrease = mostdecrease[mostdecrease["Decrease_%"] > 0].reset_index(drop=True)

# tampilkan produk pada kategori "Others" yang mengalami penurunan
display(mostdecrease)

In [ ]:
# visualisasikan data dengan bar chart
plt.figure(figsize=(14,6))
sns.barplot(
    data=mostdecrease,
    x="sku_name",
    y="Decrease_%",
    color="salmon"
)

plt.xticks(rotation=75, ha="right")
plt.title("Produk dengan Penurunan Terbesar 2021 sd 2022 Kategori 'Others'")
plt.xlabel("Nama Produk")
plt.ylabel("Penurunan Penjualan (%)")
plt.tight_layout()
plt.show() # hasil produk yang mengalami penurunan pada kategori "Others"

Analisa 3
---
Analisa untuk melihat customer yang sudah check-out tetapi belum melakukan pembayaran (keterangan pada kolom: is_gross = 1) pada tahun 2022, kemudian diambil datanya berupa Customer ID dan Registered Date nya.

In [ ]:
# Filter dasar
df_filtered = df[(df['is_gross'] == 1) & # ini sudah melakukan check-out
                 (df['is_valid'] == 0) & # tetapi belum melakukan pembayaran
                 (df['is_net'] == 0) & # memastikan transaksi belum tercatat
                 (df['order_date'].dt.year == 2022)] # filter tahun 2022

df_filtered = df_filtered[['customer_id', 'registered_date']] # Ambil kolom yang diminta

df_filtered.head(5) # menampilkan data yang telah dilakukan filter

In [ ]:
# Download file CSV data yang dibutuhkan
df_filtered.to_csv('customer sudah checkout tapi belum bayar.csv', encoding = 'utf-8-sig',index=False)
files.download('customer sudah checkout tapi belum bayar.csv')

Analisa 4
---
Analisa penjualan pada bulan Oktober sd Desember 2022 pada produk promosi transaksi yang mendapatkan diskon (kondisi kolom, discount_amount = >0) memiliki kenaikan penjualan atau tidak.
Membuat dan melihat perbandingan pada penjualan rata-rata harian weekend (Sabtu dan Minggu) vs rata-rata harian penjualan weekday (Senin-Jumat) selama Oktober sd Desember 2022 dan juga keseluruhan selama Oktober sd Desember 2022.

In [ ]:
# Bikin data dengan jumlah maks dua angka dibelakang desimal
pd.options.display.float_format = '{:.2f}'.format

# Membuat kolom
df['day'] = df['order_date'].dt.day_name() # kolom untuk hari
df['month'] = df['order_date'].dt.month_name() # kolom untuk bulan
df['month_num'] = df['order_date'].dt.month # kolom untuk nomor bulan

# Filter dasar
df_filtered = df[df['is_valid'] == 1] # filter yang sudah dibayar
df_campaign = df_filtered[(df_filtered['order_date'].dt.year == 2022) & (df_filtered['order_date'].dt.month.isin([10, 11, 12]))]

# Memisahkan antara weekend dan weekday
df_weekends = df_campaign[df_campaign['day'].isin(['Saturday', 'Sunday'])]
df_weekdays = df_campaign[df_campaign['day'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])]

In [ ]:
# Rata-rata penjualan perbulan
weekend_sum_by_month = df_weekends.groupby(['month_num', 'month'])['before_discount'].sum()
weekend_days_by_month = df_weekends.groupby(['month_num', 'month'])['order_date'].nunique()

weekday_sum_by_month = df_weekdays.groupby(['month_num', 'month'])['before_discount'].sum()
weekday_days_by_month = df_weekdays.groupby(['month_num', 'month'])['order_date'].nunique()

average_sales_month = pd.DataFrame({
    'Weekend Average': weekend_sum_by_month.div(weekend_days_by_month, fill_value=0), # .div agar aman dari pembagian 0
    'Weekday Average': weekday_sum_by_month.div(weekday_days_by_month, fill_value=0)
})

# Display result table
display(average_sales_month)

In [ ]:
# Rata-rata pada 3 bulan (Oktober, November, Desember)
avg_weekend_3bulan = average_sales_month['Weekend Average'].mean()
avg_weekday_3bulan = average_sales_month['Weekday Average'].mean()

summary_avg = pd.DataFrame({
    'Type': ['Weekend', 'Weekday'],
    'Average Oct - Des': [avg_weekend_3bulan, avg_weekday_3bulan]
})

# Display result table
display(summary_avg)

In [ ]:
# Visualisasikan dalam bentuk bar chart

# Visualisasi data dalam rata - rata perbulan
avg_plot = average_sales_month.reset_index()
x = np.arange(len(avg_plot['month']))
width = 0.35
plt.figure(figsize=(10,6))

plt.bar(x - width/2, avg_plot['Weekend Average'], width, label='Weekend Average', color='orange')
plt.bar(x + width/2, avg_plot['Weekday Average'], width, label='Weekday Average', color='blue')

plt.xticks(x, avg_plot['month'])
plt.title("Perbandingan Rata-rata Harian Weekend vs Weekday (Okt–Des 2022)")
plt.xlabel("Month")
plt.ylabel("Average Daily Sales")
plt.legend()
plt.tight_layout()
plt.show()

# Visualisasi data dalam rata - rata dalam 3 bulan
plt.figure(figsize=(6,4))
plt.bar(summary_avg['Type'], summary_avg['Average Oct - Des'])

plt.title("Rata-rata Penjualan Weekend vs Weekday (Okt–Des)")
plt.ylabel("Average Sales")
plt.show()
